Nama:

In [1]:
print("Rahman Aziz Firmansyah")

Rahman Aziz Firmansyah


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, OneHotEncoder, PolynomialFeatures
from sklearn.feature_selection import mutual_info_classif, SelectKBest, VarianceThreshold, mutual_info_regression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, precision_recall_curve, roc_auc_score, roc_curve, r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
from surprise import KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split as tts_surprise
from surprise.model_selection import GridSearchCV as gscv_surprise

(1) Buatlah sistem rekomendasi popularity-based berupa fungsi kustom yang menerima 3 argumen: dataset, jumlah lagu yang direkomendasikan, dan minimal jumlah rating lagu. Print rekomendasi top 10 lagu.

In [3]:
song = pd.read_csv('song.csv')
song.head()

,song_id,title,artist_name,user_id,rating
0,5076,Eli_ The Barrow Boy,The Decemberists,30266,1.0
1,5076,Eli_ The Barrow Boy,The Decemberists,66345,1.0
2,5076,Eli_ The Barrow Boy,The Decemberists,64781,1.0
3,5076,Eli_ The Barrow Boy,The Decemberists,16079,1.0
4,5076,Eli_ The Barrow Boy,The Decemberists,23987,1.0


In [91]:
def recomm_song(data, n_songs=5, threshold=100):
    #menyimpan data id, judul, average ratings, dan berapa kali lagu tersebut diulas
    song_id = {}
    id_rate = {}
    id_count = {}
    for id in data['song_id'].unique():
        song_id[id] = data['title'][data['song_id'] == id].unique()[0]
        id_rate[id] = np.mean(data['rating'][data['song_id'] == id])
        id_count[id] = len(data[data['song_id'] == id])
    #membuat dataframe dari dictionary yang telah dibuat dan disorting dari count_rate tertinggi ke terendah
    df_sorted = pd.DataFrame({'id': song_id.keys(), 'title': song_id.values(), 'avg_rate': id_rate.values(),
                         'count_rate': id_count.values()}).sort_values(by='count_rate', ascending=False).reset_index(drop=True)
    df_disp = df_sorted[df_sorted['count_rate'] >= threshold].iloc[:n_songs, :]
    return df_disp

In [92]:
recomm_song(song)

,id,title,avg_rate,count_rate
0,8612,Fix You,1.152271,2246
1,3165,Mockingbird,1.186603,1881
2,3564,Fuck Kitty,1.446466,1429
3,4160,How You Remind Me,1.178909,1375
4,4289,Do You Remember,1.155068,1322


In [93]:
recomm_song(song, 7)

,id,title,avg_rate,count_rate
0,8612,Fix You,1.152271,2246
1,3165,Mockingbird,1.186603,1881
2,3564,Fuck Kitty,1.446466,1429
3,4160,How You Remind Me,1.178909,1375
4,4289,Do You Remember,1.155068,1322
5,7977,Speed Of Sound,1.146058,1205
6,8434,Howlin For You,1.201566,1022


In [94]:
recomm_song(song, 10, 1300)

,id,title,avg_rate,count_rate
0,8612,Fix You,1.152271,2246
1,3165,Mockingbird,1.186603,1881
2,3564,Fuck Kitty,1.446466,1429
3,4160,How You Remind Me,1.178909,1375
4,4289,Do You Remember,1.155068,1322


(2) Buatlah sistem rekomendasi collaborative filtering user-user. Hitung RMSE dari sistem rekomendasi yang dibuat. Buat sistem rekomendasi berupa fungsi kustom yang menerima 4 argumen: dataset, user id, jumlah lagu yang direkomendasikan, dan algoritma. Pastikan rekomendasi yang diberikan adalah lagu yang belum diberi rating oleh user. Print rekomendasi top 10 lagu untuk salah satu user.

In [8]:
song_quarter = pd.read_csv('song_quarter.csv')
song_quarter.head()

,song_id,title,artist_name,user_id,rating
0,3165,Mockingbird,Eminem,20852,1.0
1,6263,Here We Go Again (Album Version),Paramore,24713,1.0
2,8518,Love Is Here,Tenth Avenue North,72102,1.0
3,5706,Mykonos,Fleet Foxes,44037,1.0
4,859,Stroke You Up (LP Version),Changing Faces,8557,1.0


In [63]:
song_quarter['title'][song_quarter['song_id'] == 3165].unique()[0]

'Mockingbird'

In [9]:
song_quarter.shape

(21736, 5)

In [10]:
song_new = song_quarter.loc[:5000, ['user_id', 'song_id', 'rating']]
song_new.shape

(5001, 3)

In [11]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(song_new, reader)
trainset, testset = tts_surprise(data, test_size=0.2, random_state=42)

In [12]:
param_grid = {'k': [20, 30, 40],
             'min_k': [3, 6, 9],
             'sim_options': {'name': ['msd', 'pearson', 'cosine'],
                             'user_based': [True]}}

gscv = gscv_surprise(KNNBasic, param_grid, cv=5, measures = ['rmse'])

In [13]:
gscv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine s

Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing s

In [14]:
knn_basic_user = KNNBasic(**gscv.best_params)

In [15]:
knn_basic_user.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [16]:
prediction = knn_basic_user.test(testset)
accuracy.rmse(prediction)

RMSE: 0.4650


0.4649892160351367

In [22]:
song_new['user_id'].unique()

array([20852, 24713, 72102, ..., 21144, 44572, 23873], dtype=int64)

In [89]:
def song_pop(data, usid, n_songs=7, algo=knn_basic_user):
    listened = data[data['user_id'] == usid]['song_id'].unique()
    unlistened = data[data['user_id'] != usid]['song_id'].unique()
    est_rating = [algo.predict(usid, song_id).est for song_id in unlistened]
    title = [song_quarter['title'][song_quarter['song_id'] == song_id].unique()[0] for song_id in unlistened]
    return pd.DataFrame({'song_id': unlistened, 
              'est_rating': est_rating, 
              'title': title}).sort_values(by='est_rating', ascending=False).iloc[:n_songs, :].reset_index(drop=True)

In [90]:
song_pop(song_new, 20852)

,song_id,est_rating,title
0,1210,2.000000,Boombox
1,8612,1.666667,Fix You
2,6263,1.187000,Here We Go Again (Album Version)
3,110,1.187000,Cancer
4,6784,1.187000,Mad World
5,9105,1.187000,Rest
6,7775,1.187000,Weenie Beenie


(3) Buatlah sistem rekomendasi collaborative filtering item-item. Hitung RMSE dari sistem rekomendasi yang dibuat. Buat sistem rekomendasi berupa fungsi kustom yang menerima 4 argumen: dataset, user id, jumlah lagu yang direkomendasikan, dan algoritma. Pastikan rekomendasi yang diberikan adalah lagu yang belum diberi rating oleh user. Print rekomendasi top 10 lagu untuk salah satu user.

In [102]:
param_grid = {'k': [20, 30, 40],
             'min_k': [3, 6, 9],
             'sim_options': {'name': ['msd', 'pearson', 'cosine'],
                           'user_based': [False]}}

gscv = gscv_surprise(KNNBasic, param_grid, cv=5, measures = ['rmse'])

In [103]:
gscv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine s

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the ms

In [104]:
knn_basic_item = KNNBasic(**gscv.best_params)

In [105]:
knn_basic_item.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [106]:
prediction = knn_basic_item.test(testset)
accuracy.rmse(prediction)

RMSE: 0.4650


0.4649892160351367

In [108]:
song_pop(song_new, 20852, algo=knn_basic_item)

,song_id,est_rating,title
0,1210,2.000000,Boombox
1,8612,1.666667,Fix You
2,6263,1.187000,Here We Go Again (Album Version)
3,110,1.187000,Cancer
4,6784,1.187000,Mad World
5,9105,1.187000,Rest
6,7775,1.187000,Weenie Beenie


In [113]:
pd.concat([song_pop(song_new, 20852, algo=knn_basic_item), song_pop(song_new, 20852, algo=knn_basic_user)], axis=1)

,song_id,est_rating,title,song_id,est_rating,title
0,1210,2.000000,Boombox,1210,2.000000,Boombox
1,8612,1.666667,Fix You,8612,1.666667,Fix You
2,6263,1.187000,Here We Go Again (Album Version),6263,1.187000,Here We Go Again (Album Version)
3,110,1.187000,Cancer,110,1.187000,Cancer
4,6784,1.187000,Mad World,6784,1.187000,Mad World
5,9105,1.187000,Rest,9105,1.187000,Rest
6,7775,1.187000,Weenie Beenie,7775,1.187000,Weenie Beenie
